<a href="https://colab.research.google.com/github/LivingstonTardzenyuy/-WorldQuant-University-Data-Science-projects/blob/main/07_miltestone_project_1_food_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Milestone Project 1: Food Vision Big.

## checkout GPU.
* Google Colab offers free GPUs, howerver, not all of them are compatiable with mixed precision training.

Google Colab offers:
* K80 (not compatible with mixed precision training)
* P100 (compatible with mixed precision training)
* T4 (compatible with mixed precision training)

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1fe89c69-4615-ad3d-88cb-2450c240208e)


## Get helper functions.

We've created helper functions in the past. Now we will call them and continue utilizing them..
https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/extras/helper_functions.py

In [2]:
# Download helper functions script.

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2025-03-03 01:44:08--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-03-03 01:44:08 (49.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# import a series of helper functions for the notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
# Get TensorFlow Datasets.
import tensorflow_datasets as tfds

In [5]:
# List all avialable datasets.
datasets_list = tfds.list_builders()
print("food101" in datasets_list)

True


In [22]:
# Load in the data (takes 5-6 minues in Google Colab).
(train_data, test_data), ds_info = tfds.load(name="food101",
                                               split=["train", "validation"],
                                               shuffle_files=True,
                                               as_supervised=True,    # data get return in tuple.
                                               with_info=True
                                             )

In [ ]:
## Features of Food101 from TFDS.
ds_info.features

In [ ]:
# Get the class names.
class_names = ds_info.features["label"].names
class_names[:10]

## Exploring the Food101 data from TensorFlow Datasets.

To become one with our data, we want to find:

* Class names
* The shape of our input data (image tensors)
* The datatype of our input data.
* What the labels look like (e.g are they one-hot encoded or are they label encoded)
* Do the labels match up with the class names ?.


In [ ]:
# Take one sample of the train data.
train_one_sample = train_data.take(1)    # samples are in format (image_tensor, label)
train_one_sample

In [ ]:
# Output infor about our training sample.
for image, label in train_one_sample:
  print(f"Image shape: {image.shape}")
  print(f"Image datatype: {image.dtype}")
  print(f"Target class from class_names: {class_names[label]}")
  print(f"Target class from label: {label}")

In [ ]:
# What does our image tensor from TFDS's Food101 look like ?.
image

In [ ]:
# What are the min and max values of our image tensor ?.
import tensorflow as tf
tf.reduce_min(image), tf.reduce_max(image)

## Plot an image from TensorFlow Datasets

In [ ]:
# Plot an image tensor
import matplotlib.pyplot as plt

plt.imshow(image)
plt.axis("off")
plt.title(class_names[label.numpy()])

# Create preprocessing functions for our data.

Neural networks perform best when are data is in a certain way. eg. Batched, normalized etc.

However, not all data (including data from TensorFlow Datasets) is in the way we want it.

So in order to get it ready for our neural network we'll have to write preprocessing function and map it with our data.

What we know about our data:

* In 'unit8' datatype.
* Comprised of all different size of tensors (different sized images)
* Not scaled (the pixel values are between 0-255)


What we know models like:
* Data in 'float32' dtype (or for mixed precision 'float16' and 'float32')
* For batches, TensorFlow likes all of the tensors within a batch to be of the same size.
* Scaled (values between 0-1) also called Normalized/scaling tensors generally perform better.

With thses points in mind, we've got a few things to tackle with a preprocesing function.

Since we've been using EfficientNetBx pretrained model from tf.keras.applications we don't need to rescale our data(these architectures have rescaling built-in).

This means our function have to

* reshape our iamges to same size.
* Convert the dtype of our image tensors from 'unit8' to float32.


In [ ]:
# Make a function for preprocessing our image.
def preprocess_img(image, label, img_shape = 224):
  """
    Converts image datatype from 'uint8' to 'float32' and reshapes image to [img_shape, img_shape, color_channels].
  """

  # reshape our image size.
  image = tf.image.resize(image, [img_shape, img_shape])

  # converting our images from unit8 to float32.
  image = tf.cast(image, tf.float32)
  # image = image/255. # Scale image values. But its not required here since EfficientNetB0 already have rescalling build in.
  return image, label

In [ ]:
# Preprocess a single sample image and check the outputs.
preprocessed_img = preprocess_img(image, label)
print(f"Image before preprocessing:\n {image[:2]}...,  \nShape: {image}, \nDatatype: {image.dtype}")
print(f"Image after preprocessing:\n {preprocessed_img[0][:2]}...,  \nShape: {preprocessed_img[0].shape}, \nDatatype: {preprocessed_img[0].dtype}")


## Batch & prepare datasets.

We're now going to make our data input pipeline run really fast.

For more resources on this: https://www.tensorflow.org/guide/data_performance

In [34]:
# Map preprocessing function to training (and parallelize)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle train_data and turn it into batches and prefetch.
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map preprocessing function to test data.
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_data, test_data

 ## Create modellling callbacks.

  We're going to create a couple of callbacks to help us while our model trains:

  * TensorBoard callback to log training results (so we can visualize them later if need be)

  * ModelCheckpoint callback to save our model progress after feature extraction.

In [36]:
# Create tensorboard callback. (inport from helper_functions.py)
from helper_functions import create_tensorboard_callback

# Create ModelCheckpoint callback to save a model's progress during training.
checkpoint_path = "model_checkpoints/cp.weights.h5"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    save_weights_only=True,
    monitor="val_accuracy",
    save_best_only=True,
    verbose=0
)


## Setup mixed precision traning.

First and foremost, for a deeper understanding of mixed precision traning. We'll see https://www.tensorflow.org/guide/mixed_precision

Mixed precision utilizes a combination of Float16 and Float32 to speed up performance.

In [37]:
# Trun on mixed precision
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy("mixed_float16")

In [38]:
model_checkpoint

## Creating the Base model(Feature Extraction)

In [39]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision

#Download base model and freeze underlying layers
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False
INPUT_SHAPE = [224, 224, 3]
# Create a functional model.
inputs = layers.Input(shape=INPUT_SHAPE)
# Note: EfficientNetBX models already have Rescaling layers build-int but if we're using a d/f model we'll neeed to include Rescalling layer here.
# x = Rescalling(1/255)(x)
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)    # We include the dtype here since we've setup mixed precision.
model = tf.keras.Model(inputs, outputs)

In [40]:
# Compile the model.
model.compile(
    loss = "sparse_categorical_crossentropy",    # We change our loss to Sparsh_categorical_crossentropy since our labels are as int.
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
    )

In [41]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 101)                 │         129,381 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ cast_4 (Cast)                        │ (None, 101)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax_float32 (Activation)         │ (None, 101)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,178,952 (15.94 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

## Checking layers dtype policies(Are we using mixed precision)

In [42]:
# check the dtype_policy.
for layer in model.layers:
  print(layer.name, layer.trainable, layer.dtype_policy)

input_layer_4 True <DTypePolicy "mixed_float16">
efficientnetb0 False <DTypePolicy "mixed_float16">
global_average_pooling2d_1 True <DTypePolicy "mixed_float16">
dense_1 True <DTypePolicy "mixed_float16">
softmax_float32 True <DTypePolicy "float32">


In [43]:
## Fitting our model.
model_history = model.fit(
    train_data,
    epochs = 5,
    steps_per_epoch = len(train_data),
    validation_data = test_data,
    validation_steps = int(0.15 * len(test_data)),
    callbacks = [create_tensorboard_callback("training_logs", "efficientnetb0_feature_extraction"), model_checkpoint]
)

Saving TensorBoard log files to: training_logs/efficientnetb0_feature_extraction/20250303-021615
Epoch 1/5
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 225s 82ms/step - accuracy: 0.4765 - loss: 2.2828 - val_accuracy: 0.6920 - val_loss: 1.1453
Epoch 2/5
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 221s 71ms/step - accuracy: 0.6659 - loss: 1.2945 - val_accuracy: 0.7158 - val_loss: 1.0451
Epoch 3/5
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 176s 73ms/step - accuracy: 0.7035 - loss: 1.1376 - val_accuracy: 0.7233 - val_loss: 0.9952
Epoch 4/5
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 190s 79ms/step - accuracy: 0.7261 - loss: 1.0390 - val_accuracy: 0.7270 - val_loss: 0.9812
Epoch 5/5
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 193s 76ms/step - accuracy: 0.7423 - loss: 0.9775 - val_accuracy: 0.7309 - val_loss: 0.9592


In [44]:
# Evaluate model on test_dataset.
results_feature_extract_model = model.evaluate(test_data)
results_feature_extract_model

790/790 ━━━━━━━━━━━━━━━━━━━━ 58s 74ms/step - accuracy: 0.7335 - loss: 0.9653


[0.9618264436721802, 0.7388119101524353]

In [48]:
# Saving our build model locally.
model_save = model.save("model_checkpoints/feature_extract_model.keras")

In [53]:
# Load our save model.
loaded_model = tf.keras.models.load_model("model_checkpoints/feature_extract_model.keras")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 8 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 4 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [54]:
# Checking the loaded model performance.
loaded_model.evaluate(test_data)

790/790 ━━━━━━━━━━━━━━━━━━━━ 63s 66ms/step - accuracy: 0.7337 - loss: 0.9650


[0.961826503276825, 0.7388119101524353]

## Preparing our model's layer for Fine-tuning

In [55]:
# Downloaded the save model from google storage.
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip

--2025-03-03 02:38:12--  https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.207, 74.125.195.207, 172.253.117.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16976857 (16M) [application/zip]
Saving to: ‘07_efficientnetb0_feature_extract_model_mixed_precision.zip’

07_efficientnetb0_f 100%[===================>]  16.19M  84.8MB/s    in 0.2s    

2025-03-03 02:38:13 (84.8 MB/s) - ‘07_efficientnetb0_feature_extract_model_mixed_precision.zip’ saved [16976857/16976857]



In [56]:
# Load and evaluate downloaded GS model.
loaded_model = tf.keras.models.load_model("07_efficientnetb0_feature_extract_model_mixed_precision.zip")
loaded_model.evaluate(test_data)

ValueError: File format not supported: filepath=07_efficientnetb0_feature_extract_model_mixed_precision.zip. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(07_efficientnetb0_feature_extract_model_mixed_precision.zip, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [57]:
# Getting a summary of the downloaded model.
loaded_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 101)                 │         129,381 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ cast_6 (Cast)                        │ (None, 101)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax_float32 (Activation)         │ (None, 101)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,308,339 (16.44 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

 Optimizer params: 129,387 (505.43 KB)

In [58]:
# Seting all the layers .trianable variable in the loaded model to True.
base_model.trainable = True

In [69]:
for layer in base_model.layers[:-10]:
  layer.trainable = False

In [70]:
# Checking the dtype_policy of the layers in our loaded model.
for layer in loaded_model.layers:
  print(layer.name, layer.trainable, layer.dtype_policy)

input_layer_4 True <DTypePolicy "mixed_float16">
efficientnetb0 False <DTypePolicy "mixed_float16">
global_average_pooling2d_1 True <DTypePolicy "mixed_float16">
dense_1 True <DTypePolicy "mixed_float16">
softmax_float32 True <DTypePolicy "float32">


In [71]:
# Setting up EarlyStopping callback to stop training.
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor="val_accuracy",
                               patience=3,
                               restore_best_weights=True
                               )

In [72]:
# Creating a modelCheckpoint callbakc to save best model during fine-tuning.
from tensorflow.keras.callbacks import ModelCheckpoint
model_checkpoint = ModelCheckpoint(
    "model_checkpoints/fine_tune_model.h5",
    save_best_only=True,
    monitor="val_loss"
)

In [73]:
# Compile our model.
loaded_model.compile(
    loss = "sparse_categorical_crossentropy",           # Since our labels are integers represinting the class index.
    optimizer = tf.keras.optimizers.Adam(0.0001),
    metrics = ["accuracy"]
)

In [76]:
# Define the create_tensorboard_callback function directly in your script
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback


In [77]:
# Fine-tune for 5 more epochs
fine_tune_epochs = 3
total_epochs = 5 + fine_tune_epochs
total_epochs

8

In [ ]:
# Fitting the fine-tune model.
fine_tune_history = loaded_model.fit(
    train_data,
    epochs = total_epochs,
    steps_per_epoch = len(train_data),
    validation_data = test_data,
    initial_epoch = model_history.epoch[-1],
    validation_steps = int(0.15 * len(test_data)),
    callbacks = [create_tensorboard_callback("training_logs", "efficientnetb0_fine_tune"), early_stopping, model_checkpoint]
)

Saving TensorBoard log files to: training_logs/efficientnetb0_fine_tune/20250303-030008
Epoch 5/8
1695/2368 ━━━━━━━━━━━━━━━━━━━━ 45s 67ms/step - accuracy: 0.7802 - loss: 0.8393

In [ ]:
# evaluate the fine-tune model.
loaded_model.evaluate(test_data)